In [ ]:
import os
import warnings

import pandas as pd

from covid_model_deaths.data import get_input_data, plot_crude_rates
from covid_model_deaths import runner

pd.options.display.max_rows = 99
pd.options.display.max_columns = 99
warnings.simplefilter('ignore')

RUN_TYPE = 'prod'
DATESTAMP_LABEL = '2020_04_17_US'
DATA_VERSION = 'best'
PEAK_FILE = '/ihme/covid-19/deaths/mobility_inputs/2020_04_14/final_peak_locs_04_14.csv'
CASES_DEATHS_FILE = '/ihme/covid-19/deaths/mobility_inputs/2020_04_14/deaths_from_cases.csv'
PEAK_DURATION_FILE = '/ihme/covid-19/deaths/mobility_inputs/2020_04_14/smooth_peak_duration.csv'
R0_FILE = '/ihme/covid-19/deaths/mobility_inputs/2020_04_14/R0_dates.csv'



CODE_DIR = os.path.abspath('../src/covid_model_deaths')
OUTPUT_DIR = f'/ihme/covid-19/deaths/{RUN_TYPE}/{DATESTAMP_LABEL}'
checkpoint = runner.Checkpoint(OUTPUT_DIR)

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
print(f'Writing to {OUTPUT_DIR}')
print(CODE_DIR)
print(checkpoint)

def filter_data(data: pd.DataFrame) -> pd.DataFrame:
    # drop Georgia the country until we fix location_ids
    data = data[data['Country/Region'] != 'Georgia'].reset_index(drop=True)

    ## 04/16 - crazy numbers in NY, drop today
    state_column = 'Province/State' if 'Province/State' in data.columns else 'Location'
    data = data.loc[~((data[state_column] == 'New York') & data['Date'] >= pd.Timestamp('2020-04-06'))]
    return data

In [ ]:
# Store pops for Bobby.
get_input_data('us_pops').to_csv(f'{OUTPUT_DIR}/pops.csv', index=False)

## read full (unrestricted) set from snapshot

In [ ]:
# come up with more informative names...
input_full_df = filter_data(get_input_data('full_data', DATA_VERSION))
input_death_df = filter_data(get_input_data('deaths', DATA_VERSION))
input_age_pop_df = get_input_data('age_pop', DATA_VERSION)
input_age_death_df = get_input_data('age_death', DATA_VERSION)


# # read in cov input file (predicted date of R0 == 1) to see if we are using these for a given location
# cov_df = pd.read_csv(R0_FILE)
# r0_locs = cov_df['location_id'].unique().tolist()
# del cov_df
r0_locs = []

# # less conservative peak ranges
# peak_dur_df = pd.read_csv(PEAK_DURATION_FILE)
# peak_dur_df = peak_dur_df.loc[peak_dur_df['Location'] != 'Colorado']
# peak_dur_df['peak start date'] = pd.to_datetime(peak_dur_df['peak start date'])
# peak_dur_df['peak end date'] = pd.to_datetime(peak_dur_df['peak end date'])

checkpoint.write('full_data', input_full_df)
checkpoint.write('deaths', input_death_df)
checkpoint.write('age_pop', input_age_pop_df)
checkpoint.write('age_death', input_age_death_df)

In [ ]:
death_df = checkpoint.load('deaths')

# plot
plot_crude_rates(death_df, level='subnat')


## combine back-casted death rates with cases for abie (using model dataset, i.e. admin1 and below)

In [ ]:
full_df = checkpoint.load('full_data')
death_df = checkpoint.load('deaths')
age_pop_df = checkpoint.load('age_pop')
age_death_df = checkpoint.load('age_death')

cases_and_backcast_deaths_df = runner.make_cases_and_backcast_deaths(full_df, death_df, age_pop_df, age_death_df)

cases_and_backcast_deaths_df.to_csv(f'{OUTPUT_DIR}/backcast_for_case_to_death.csv', index=False)
checkpoint.write('cases_and_backcast_deaths', cases_and_backcast_deaths_df)

## Impute death thresholds.

In [ ]:
cases_and_backcast_deaths_df = checkpoint.load('cases_and_backcast_deaths')

in_us = cases_and_backcast_deaths_df['Country/Region'] == 'United States of America'
state_level = ~cases_and_backcast_deaths_df['Province/State'].isnull()
us_states = cases_and_backcast_deaths_df.loc[in_us & state_level, 'Province/State'].unique().to_list()

us_threshold_dates = runner.impute_death_threshold(cases_and_backcast_deaths_df,
                                                   location_list=us_states)
us_threshold_dates = cases_and_backcast_deaths_df.merge(us_threshold_dates)
us_threshold_dates.to_csv(f'{OUTPUT_DIR}/threshold_dates.csv', index=False)
checkpoint.write('threshold_dates', us_threshold_dates)


## Make last day data

In [ ]:
full_df = checkpoint.load('full_data')
us_threshold_dates = checkpoint.load('threshold_dates')

us_date_mean_df = runner.make_date_mean_df(us_threshold_dates)
last_day_df = runner.make_last_day_df(full_df, us_date_mean_df)

checkpoint.write('date_mean', us_date_mean_df)
checkpoint.write('last_day', last_day_df)

## Submit models

In [ ]:
full_df = checkpoint.load('full_data')
death_df = checkpoint.load('deaths')
age_pop_df = checkpoint.load('age_pop_df')
us_date_mean_df = checkpoint.load('date_mean')
last_day_df = checkpoint.load('last_day')

cases_deaths_df = pd.read_csv(CASES_DEATHS_FILE)
cases_deaths_df['Date'] = pd.to_datetime(cases_deaths_df['Date'])

us_location_ids, us_location_names = runner.get_us_location_ids_and_names(full_df)

submodel_dict = runner.submit_models(full_df, death_df, age_pop_df, age_death_df, us_date_mean_df, cases_deaths_df,
                                     us_location_ids, us_location_names, r0_locs,
                                     PEAK_FILE, OUTPUT_DIR, DATA_VERSION, R0_FILE, CODE_DIR)


checkpoint.write('submodel_dict', submodel_dict)

## compile draws

In [ ]:
full_df = checkpoint.load('full_data')
age_pop_df = checkpoint.load('age_pop')
us_threshold_dates = checkpoint.load('threshold_dates')
submodel_dict = checkpoint.load('submodel_dict')

us_location_ids, us_location_names = runner.get_us_location_ids_and_names(full_df)

in_us = full_df['Country/Region'] == 'United States of America'
state_level = ~full_df['Province/State'].isnull()
usa_obs_df = full_df[in_us & state_level]

draw_dfs, past_draw_dfs, models_used, days, ensemble_draws_dfs = runner.compile_draws(us_location_ids,
                                                                                      us_location_names,
                                                                                      submodel_dict,
                                                                                      usa_obs_df,
                                                                                      us_threshold_dates,
                                                                                      age_pop_df)

if 'location' not in models_used:
    raise ValueError('No location-specific draws used, must be using wrong tag')
draw_df = pd.concat(draw_dfs)
model_type_df = pd.DataFrame({'location': us_location_names,
                              'model_used': models_used})

# write
draw_df.to_csv(f'{OUTPUT_DIR}/state_data.csv', index=False)
model_type_df.to_csv(f'{OUTPUT_DIR}/state_models_used.csv', index=False)
ensemble_plot_path = runner.make_and_save_draw_plots(OUTPUT_DIR, us_location_ids, us_location_names,
                                                     ensemble_draws_dfs, days, models_used, age_pop_df)
print(ensemble_plot_path)
checkpoint.write('draw_data', draw_df)

## total US deaths in this run

In [ ]:
draw_df = checkpoint.load('draw_data')

runner.display_total_deaths(draw_df)

## combine with previous predictions

In [ ]:
# ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##
# raw_draw_path = f'{OUTPUT_DIR}/state_data.csv'
# average_draw_path = f'{OUTPUT_DIR}/past_avg_state_data.csv'
# yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_04_10_US/state_data.csv'
# before_yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_04_09_US/state_data.csv'
# ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##

# avg_draw_df = runner.average_draws(OUTPUT_DIR, 'US', raw_draw_path, yesterday_draw_path, before_yesterday_draw_path)

# ## NO NEED TO DO THIS, FOR NOW ##
# # avg_df = get_peak_date(past_draw_df, avg_df)

# # store data
# avg_df.to_csv(average_draw_path, index=False)
# print(average_draw_path)

# # plot
# compare_average_plot_path = runner.make_and_save_compare_average_plots(OUTPUT_DIR,
#                                                                        raw_draw_path,
#                                                                        average_draw_path,
#                                                                        yesterday_draw_path,
#                                                                        before_yesterday_draw_path,
#                                                                        'United States of America')
# print(compare_average_plot_path)


In [ ]:
raw_draw_path = f'{OUTPUT_DIR}/state_data.csv'
previous_draw_path = '/ihme/covid-19/deaths/prod/2020_04_16_US/state_data.csv'

compare_to_previous_plot_path = runner.make_and_save_compare_to_previous_plots(OUTPUT_DIR, raw_draw_path, previous_draw_path)


In [ ]:

viz_dir = runner.send_plots_to_diagnostics(DATESTAMP_LABEL,
                                           ensemble_plot_path,
                                           # compare_average_plot_path,
                                           compare_to_previous_plot_path)
print(viz_dir)

